In [1]:
# import pandas as pd
# df = pd.read_csv('data/processed_mean_bitter_train.csv')
# # Map the Taste column
# df["Taste"] = df["Taste"].map({"Sweet": 2, "Bitter": 1}).fillna(0).astype(int)

# # Save the modified CSV
# df.to_csv("data/BitterSweet.csv", index=False)

# print("Taste column updated successfully!")


In [2]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, roc_auc_score,
    precision_score, recall_score, f1_score, average_precision_score
)
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, roc_auc_score,
    precision_score, recall_score, f1_score, average_precision_score
)
import numpy as np



In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest, RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from boruta import BorutaPy
from joblib import dump, load
bitter_features = pd.read_csv('data/BitterSweet.csv')
bitter_features = bitter_features.iloc[:2235]
replaced_data_mean = bitter_features.iloc[:, 7:]

y = bitter_features['Taste']

# Features
X = replaced_data_mean

In [4]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, classification_report, roc_auc_score,
    precision_score, recall_score, f1_score, average_precision_score
)
import numpy as np

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
import pickle
file = open('bitter/bitter_boruta.p','rb')

boruta = pickle.load(file)

In [6]:
X_transformed = X_test.reindex(columns=boruta, fill_value=0).values

In [7]:
file = open('bitter/mean.p','rb')
import pickle
best_model =pickle.load(file)

y_pred_bitter = best_model.predict(X_transformed)
y_pred_proba_bitter = best_model.predict_proba(X_transformed)[:, 1]
print("Bitter : ")
print(y_pred_bitter)
print(y_pred_proba_bitter)


Bitter : 
[False  True False False  True False  True  True  True False  True  True
 False  True False  True  True False False False False False False False
 False  True False False False False False  True  True False False False
 False  True False  True  True False False  True False False  True False
 False False  True  True False False  True False  True False  True False
 False False False  True  True False  True False False  True False False
 False False False False  True False False False  True False False False
  True False  True  True False False False False False False False False
 False False False False False  True  True  True False False False False
  True False False False  True False False  True False  True False  True
 False False False False False False False False  True  True False False
  True  True False False  True False False False False  True False False
  True False False  True False False False False  True False False  True
 False False False False  True False Fals

In [8]:
import pickle
file = open('sweet/sweet_boruta.p','rb')

boruta_sweet = pickle.load(file)

In [9]:
X_transformed_sweet = X_test.reindex(columns=boruta_sweet, fill_value=0).values

In [10]:
file = open('sweet/mean.p','rb')
import pickle
best_model =pickle.load(file)

y_pred_sweet = best_model.predict(X_transformed_sweet)
y_pred_proba_sweet = best_model.predict_proba(X_transformed_sweet)[:, 1]
print("Sweet : ")
print(y_pred_sweet)
print(y_pred_proba_sweet)


Sweet : 
[ True False False  True False  True False False False  True False False
  True False  True False False  True  True False  True  True  True  True
 False False  True  True  True  True  True False False False  True  True
  True False False False False  True  True False  True False False  True
  True  True False False  True  True False False False  True False  True
 False False  True False False  True False  True False False  True  True
 False  True  True  True False  True  True  True False  True  True  True
 False False False False False  True  True  True False  True  True False
  True  True  True False  True False False False  True  True  True False
 False  True  True  True False  True  True False  True False  True False
  True  True  True False  True  True False  True False False  True False
 False False  True  True False  True False  True  True False  True  True
 False  True  True False False  True  True  True False  True  True False
  True  True  True  True False  True  True

In [11]:
# for i in range(2000):
#     print(y[i]," ",y_pred_bitter[i]," ",y_pred_sweet[i],' ',y_pred_proba_bitter[i]," ",y_pred_proba_sweet[i])

In [12]:
import numpy as np
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report,
    average_precision_score, confusion_matrix
)

# Convert predictions into a single class label (tasteless: 0, bitter: 1, sweet: 2)
y_pred = []
for bitter, sweet, bittprob, sweetprob in zip(y_pred_bitter, y_pred_sweet, y_pred_proba_bitter,y_pred_proba_sweet):
    if bitter & sweet : 
        if bittprob > sweetprob:
            y_pred.append(1)
        else : 
            y_pred.append(2)
    elif bitter:
        y_pred.append(1)  # Bitter
    elif sweet:
        y_pred.append(2)  # Sweet
    else:
        y_pred.append(0)  # Tasteless

# Convert probability predictions into a single class probability (ensuring they sum to 1)
y_pred_proba = np.array([
    [1 - (p_bitter + p_sweet), p_bitter, p_sweet] for p_bitter, p_sweet in zip(y_pred_proba_bitter, y_pred_proba_sweet)
])

# Ensure numerical stability: clip probabilities between [0,1] and normalize
y_pred_proba = np.clip(y_pred_proba, 0, 1)
y_pred_proba /= y_pred_proba.sum(axis=1, keepdims=True)  # Normalize to sum to 1

# Compute Metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
recall = recall_score(y_test, y_pred, average='macro', zero_division=0)
f1 = f1_score(y_test, y_pred, average='macro', zero_division=0)
roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovr')  # Fixed by ensuring probabilities sum to 1
aupr = average_precision_score(y_test, y_pred_proba, average='macro')
classification_rep = classification_report(y_test, y_pred, zero_division=0)

# Confusion matrix to compute specificity and sensitivity
conf_matrix = confusion_matrix(y_test, y_pred)
tn = conf_matrix[0, 0]  # True Negatives for tasteless
fp = conf_matrix[0, 1:].sum()  # False Positives for tasteless
fn = conf_matrix[1:, 0].sum()  # False Negatives for tasteless
tp = conf_matrix[1:, 1:].sum()  # True Positives for bitter and sweet

specificity = tn / (tn + fp) if (tn + fp) > 0 else 0  # True Negative Rate
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0  # True Positive Rate (Recall)
non_error_rate = (tp + tn) / conf_matrix.sum()  # Non-error rate

# Print Results
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall (Sensitivity): {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC-AUC Score: {roc_auc:.4f}")
print(f"AUPR Score: {aupr:.4f}")
print(f"Non Error Rate: {non_error_rate:.4f}")
print(f"Specificity: {specificity:.4f}")
print("\nClassification Report:\n", classification_rep)


Accuracy: 0.9463
Precision: 0.8840
Recall (Sensitivity): 0.9329
F1-score: 0.9035
ROC-AUC Score: 0.9934
AUPR Score: 0.9671
Non Error Rate: 0.9530
Specificity: 0.9048

Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.90      0.78        42
           1       0.97      0.93      0.95       158
           2       0.99      0.96      0.98       247

    accuracy                           0.95       447
   macro avg       0.88      0.93      0.90       447
weighted avg       0.95      0.95      0.95       447

